# Kernel Regression

## Loading Packages

In [1]:
include("jlFiles/printmat.jl")
include("jlFiles/excise.jl")
include("jlFiles/lagnPs.jl")
include("jlFiles/OlsFn.jl")

OlsFn (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

## Loading Data

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',header=true)
xx  = xx[1]
ymd = xx[:,1]             #[YearMonthDay]
R   = xx[:,11]            #returns for the size portfolio we want to study

xx  = excise([R lagnPs(R)])             #return and lagged return: dependent variable and regressor
y   = xx[:,1]
x   = xx[:,2]
T   = size(x,1)
xx = nothing

println("Sample size: $T")

Sample size: 8324


A kernel regression is of the form
$$
\hat{b}(x)=\frac{\sum\nolimits_{t=1}^{T}w_{t}(x)y_{t}}{\sum\nolimits_{t=1}^{T}w_{t}(x)},
$$
where $w_{t}(x)$ is the weight of observation $t$, defined by a kernel function.

The function KernRegNormalFn uses a normal kernel, $N(x,h^{2})$.

In [4]:
function KernRegNormalFn(y,x,xGrid,h,vv)

  Ngrid = length(xGrid)                  #number of grid points

  bHat = fill(NaN,Ngrid)                 #y(t) = b[x(t)] + e(t)

  for i = 1:Ngrid                        #loop over elements in xGrid
    zi      = (x - xGrid[i])/h
    w       = exp.(-zi.^2/2)./(h*sqrt(2*pi))       #gaussian kernel, with "std" = h
    bHat[i] = sum(w[vv].*y[vv])/sum(w[vv])        #sum over observations (data)
  end

  return bHat

end

KernRegNormalFn (generic function with 1 method)

In [5]:
xGrid = collect(-10:0.25:10)
h     = 1.5
bHat    = KernRegNormalFn(y,x,xGrid,h,1:T)         #baseline choice of h
bHatHih = KernRegNormalFn(y,x,xGrid,h*2,1:T)       #high h
bHatLoh = KernRegNormalFn(y,x,xGrid,h*0.5,1:T)     #low h


ha = plot(xGrid,bHatLoh,line=(:dot,2),label="low h")
plot!(xGrid,bHat,label="medium h")
plot!(xGrid,bHatHih,line=(:dash,2),label="high h")
plot!(xlims=(minimum(xGrid),maximum(xGrid)),ylims=(-5,5))
title!("Return vs lagged return, kernel regression")
xlabel!("Lagged return")
ylabel!("Return")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 Return vs lagged return, kernel regression 
 
 
 Lagged return 
 
 
 Return 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.6; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 54.7703,42.5129 61.5365,43.9501 68.3027,45.7742 75.0688,48.1517 81.835,51.3356 88.6011,55.6817 95.3673,61.6054 102.133,69.3804 108.9,78.7073 115.666,88.3494 
 122.432,96.6974 129.198,103.358 135.964,109.905 142.73,118.764 149.497,131.621 156.263,148.389 163.029,166.766 169.795,182.835 176.561,193.163 183.327,197.042 
 190.094,196.521 196.86,194.468 203.626,192.854 210.392,192.322 217.158,192.617 223.924,193.244 230.69,193.865 237.457,194.352 244.223,194.692 250.989,194.904 
 257.755,195.021 264.521,195.078 271.287,195.111 278.054,195.138 284.82,195.167 291.586,195.195 298.352,195.218 305.118,195.234 311.884,195.239 318.651,195.234 
 325.417,195.214 332.183,195.178 338.949,195.126 345.715,195.06 352.481,194.984 359.247,194.902 366.014,194.808 372.78,194.68 379.546,194.476 386.312,194.138 
 393.078,193.614 399.844,192.898 406.611,192.083 413.377,191.428 420.143,191.377 426.909,192.469 433.675,195.076 440.441,199.136 447.208,204.081 453.974,208.965 
 460.74,212.665 467.506,214.04 474.272,212.17 481.038,206.661 487.804,197.871 494.571,186.778 501.337,174.617 508.103,162.705 514.869,152.777 521.635,147.766 
 528.401,152.424 535.168,171.791 541.934,205.545 548.7,243.98 555.466,274.953 562.232,292.587 568.998,296.196 575.765,287.076 582.531,270.224 589.297,254.08 
 596.063,243.453 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,68.6633 61.5365,72.3309 68.3027,76.5495 75.0688,81.3983 81.835,86.9558 88.6011,93.2903 95.3673,100.447 102.133,108.429 108.9,117.178 115.666,126.553 
 122.432,136.314 129.198,146.134 135.964,155.617 142.73,164.362 149.497,172.028 156.263,178.399 163.029,183.415 169.795,187.166 176.561,189.84 183.327,191.67 
 190.094,192.882 196.86,193.667 203.626,194.17 210.392,194.494 217.158,194.705 223.924,194.846 230.69,194.943 237.457,195.011 244.223,195.061 250.989,195.097 
 257.755,195.123 264.521,195.142 271.287,195.155 278.054,195.163 284.82,195.167 291.586,195.168 298.352,195.165 305.118,195.16 311.884,195.151 318.651,195.14 
 325.417,195.125 332.183,195.107 338.949,195.085 345.715,195.059 352.481,195.029 359.247,194.994 366.014,194.953 372.78,194.906 379.546,194.852 386.312,194.794 
 393.078,194.73 399.844,194.665 406.611,194.601 413.377,194.548 420.143,194.514 426.909,194.513 433.675,194.563 440.441,194.682 447.208,194.889 453.974,195.193 
 460.74,195.591 467.506,196.055 474.272,196.529 481.038,196.93 487.804,197.155 494.571,197.109 501.337,196.73 508.103,196.026 514.869,195.092 521.635,194.119 
 528.401,193.391 535.168,193.248 541.934,194.054 548.7,196.128 555.466,199.662 562.232,204.63 568.998,210.71 575.765,217.288 582.531,223.574 589.297,228.83 
 596.063,232.591 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:1.6; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 54.7703,182.363 61.5365,184.225 68.3027,185.847 75.0688,187.252 81.835,188.463 88.6011,189.502 95.3673,190.39 102.133,191.146 108.9,191.787 115.666,192.329 
 122.432,192.786 129.198,193.171 135.964,193.494 142.73,193.765 149.497,193.991 156.263,194.181 163.029,194.339 169.795,194.471 176.561,194.58 183.327,194.672 
 190.094,194.747 196.86,194.81 203.626,194.862 210.392,194.905 217.158,194.94 223.924,194.969 230.69,194.992 237.457,195.011 244.223,195.026 250.989,195.038 
 257.755,195.047 264.521,195.054 271.287,195.059 278.054,195.062 284.82,195.064 291.586,195.065 298.352,195.064 305.118,195.063 311.884,195.06 318.651,195.057 
 325.417,195.053 332.183,195.049 338.949,195.044 345.715,195.039 352.481,195.0

## Rule of Thumb Choice of h

Run the regression 

$
y=\alpha+\beta x+\gamma x^{2}+\varepsilon
$

and use the following rule-of-thumb choice

$
h  =T^{-1/5}|\gamma|^{-2/5}\sigma_{\varepsilon}^{2/5}(x_{\max}-x_{\min})^{1/5}\times0.6.
$

In practice, replace $x_{\max}-x_{\min}$ by the difference between the 90th
and 10th percentiles of $x$.

In [6]:
(b,res,) = OlsFn(y,[x.^2 x ones(T)])             #rule of thumb bandwidth

sigma = std(res)
gamm  = b[1]
xSort = sort(x)
x_10  = xSort[floor(Int,T*0.1)]          #crude 10th and 90th percentiles
x_90  = xSort[floor(Int,T*0.9)]

h_crude = 0.6*sigma^(2/5)*abs(gamm)^(-2/5)*(x_90-x_10)^(1/5)*T^(-1/5)

printlnPs("\nRule-of-thumb value of h: ",h_crude)


Rule-of-thumb value of h:      0.667


## Cross-Validation (leave-one-out)

To do a cross-validation 

(1) Pick a bandwidth $h$, do the kernel regression
but leave out observation $t$ and then record the out-of-sample prediction error $y_{t}-\hat{b}_{-t}
(x_{t},h)$. Notice that this is the error for observation $t$ only. 

(2) Repeat for all $t=1-T$ to calculate the EPE

$
\text{EPE}(h)=\sum\nolimits_{t=1}^{T}\left[  y_{t}-\hat{b}_{-t}(x_{t},h)\right]  ^{2}/T,
$

(3) Finally, redo for several different values of $h$---and pick the one
that minimizes EPE$(h)$. 

In [7]:
println("\nCross-validation calculations take some time")

hM = h_crude*[0.25 0.5 0.75 1 1.5 2 3 4 5 10]'

Nh   = length(hM)
EPEM = fill(NaN,(T,Nh))
for t = 1:T
    v_No_t = setdiff(1:T,t)
    for j = 1:Nh
        b_t       = KernRegNormalFn(y,x,x[t],hM[j],v_No_t)   #fitted b[x(t)]
        EPEM[t,j] = (y[t] - b_t[1])^2
    end
end
EPE = mean(EPEM,1)'
println("h and EPE:")
printmat([hM EPE])


Cross-validation calculations take some time
h and EPE:
     0.167       NaN
     0.334     1.314
     0.501     1.302
     0.667     1.295
     1.001     1.288
     1.335     1.286
     2.002     1.287
     2.670     1.291
     3.337     1.292
     6.674     1.296



In [8]:
ha = plot(hM,EPE/minimum(EPE),legend=false)
plot!(xlims=(minimum(hM)-0.05,maximum(hM)+0.05),ylims= (0.99,1.025))
title!("Cross validation simulations, kernel regression")
xlabel!("Bandwidth")
ylabel!("EPE/minimum(EPE)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 0.99 
 
 
 1.00 
 
 
 1.01 
 
 
 1.02 
 
 
 Cross validation simulations, kernel regression 
 
 
 Bandwidth 
 
 
 EPE/minimum(EPE)